In [21]:
import pandas as pd

data = open("/content/output.csv","r", encoding='utf-8').read()
#print(data[:500])

In [22]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.classify import ClassifierI
from statistics import mode
import pickle


In [23]:
!pip install TurkishStemmer
!pip install zemberek-python
from nltk.corpus import stopwords
nltk.download('stopwords')
import random
import re
from TurkishStemmer import TurkishStemmer
from zemberek import (
    TurkishSentenceNormalizer,
    TurkishMorphology,
    TurkishTokenizer,
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
tokenizer = TurkishTokenizer.DEFAULT
turkish_stopwords = stopwords.words("turkish")
stemmer = TurkishStemmer()
morphology = TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)

INFO:zemberek.morphology.turkish_morphology:TurkishMorphology instance initialized in 29.653531789779663


2024-12-29 14:35:53,517 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 29.653531789779663



In [25]:
import nltk
import os
nltk.download('punkt_tab')

pattern = r"\s+[^\sa-zA-Z0-9ğüşöçıĞÜŞİÖÇ,]"
document = re.sub(pattern, "", data)
print(document[:100])
lines = document.split("\n")
document = document.replace(lines[0] + '\n', '', 1)
#print(document)

Command, Label
Çizgi formasyonunu yap,Cizgi
V çiz,V
Ok başı yapıver,Ok başı
Üçgen olun,Üçgen
Çizgi ç


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [26]:
documents = []
all_words = []
for r in document.split("\n"):
  #print(r)
  if "," in r:
    command_part, label = r.rsplit(",", 1)
    #print(command_part, label)
    pattern_new = r"[^\sa-zA-Z0-9ğüşöçıĞÜŞİÖÇ]"
    command_part = re.sub(pattern_new, "", command_part)
    #print(command_part)
    command_part = normalizer.normalize(command_part)
    #print(command_part)
    command_part = command_part.lower()
    documents.append((command_part, label))
    #print(documents)
    tokens = tokenizer.tokenize(command_part)
    words = [
        stemmer.stem(token.content)
        for token in tokens
        if token.content not in turkish_stopwords
    ]
    #print(words)
    all_words.extend(words)
    #print(all_words)
    #break

random.shuffle(documents)

In [27]:
from nltk.probability import FreqDist

all_words = FreqDist(all_words)
print(all_words)
word_features = list(all_words.keys())[:400]
print(word_features)

<FreqDist with 477 samples and 2231 outcomes>
['çizg', 'formasyon', 'yap', 'v', 'çiz', 'ok', 'baş', 'yapıversen', 'üçgen', 'ol', 'oluş', 'şekl', 'çiziver', 'halin', 'gel', 'bur', 'üç', 'köşe', 'şekil', 'oluştur', 'acel', 'oluşturacak', 'hiza', 'oynay', 'voleybol', 'top', 'oyn', 'voleybolç', 'gerçekleştirme', 'gerekiyor', 'oynama', 'istirham', 'ediyor', 'ters', 'bir', 'sür', 'yapıl', 'oluşturul', 'çizs', 'drönelar', 'yapar', 'oldukç', 'müteşekkir', 'olur', 'oluşturma', 'emrediyor', 'yapma', 'ders', "drone'ler", 'ols', 'oyna', 'sürü', 'oynat', 'hadi', 'oynayal', 'iha', 'kullanarak', 'kenar', 'geometrik', 'çizdirme', 'sağl', 'güzel', 'don', 'düz', 'hat', 'elin', 'ak', 'dümt', 'ban', 'güç', 'zekan', 'harfi', 'attık', 'düş', 'dronelar', 'üçl', 'çektirmek', 'maksat', 'şekillen', 'şov', 'gösteri', 'başl', 'oyuncu', 'taklit', 'edecek', 'yarat', 'harfin', 'tasar', 'baka', 'geç', 'mot', 'maç', 'form', 'basiciz', 'uygu', 'düzg', 'çek', 'gerçekleş', 'çabuk', 'hemen', 'lan', 'çalış', 'can', 'turnuv

In [28]:
def find_features(command):

  command = normalizer.normalize(command)
  tokens = tokenizer.tokenize(command)
  words = [
      stemmer.stem(token.content)
      for token in tokens
      if token.content not in turkish_stopwords
  ]
  features = {}

  for w in word_features:
    features[w] = (w in words)

  return features

In [29]:
featuresets = [(find_features(command_part), label) for (command_part, label) in documents]
command_with_label = [(command_part, label) for (command_part, label) in documents]

In [30]:
train_size = int(len(featuresets) * 0.80)
training_test = featuresets[:train_size]
testing_set = featuresets[train_size:]
testing_set_special = command_with_label[train_size:]
classifier = nltk.NaiveBayesClassifier.train(training_test)
classifier.train(training_test)

i = 0
for test_command in testing_set_special:
  testing_state = [testing_set[i]]
  accuracy = (nltk.classify.accuracy(classifier, testing_state)) * 100

  if accuracy < 50:
        print(
            f"command: {test_command[0]} predict label: {classifier.classify(find_features(test_command[0]))}  real label: {test_command[1]}"
        )
  i += 1

new_data_ = "üç kenarlı bir şekil çizer misin."
new_data = word_tokenize(new_data_)

command: dröneları düz çizgi üzerine sırayla köycizgimisyonun drönelar ile v şekli yapmaktır sana güveniyorumvdiriliş ertuğrulun okunun başı gibi bir formasyon oluşturasın tez vakitteok başıüçgen diye bir şekil var ya onun şeklini oluştur predict label: Cizgi  real label: Üçgen
command: donlar safları sıklaştırsın predict label: voleybol  real label: Cizgi
command: bir de ters v gelir mi predict label: V  real label: Ok başı
command: dik büyüktür işareti oluştur predict label: Cizgi  real label: Ok başı
command: sana ters v formasyonunu çizmeni uygun gördümok başıyapacaksın üçgen formasyonunu predict label: V  real label: Üçgen
command: v formasyonunun ters halini al predict label: V  real label: Ok başı
command: voleybola hazır hale geçin predict label: Cizgi  real label: voleybol
command: v çizsin donlar predict label: Ok başı  real label: V


In [31]:
classifier = nltk.NaiveBayesClassifier.train(training_test)
prediction = classifier.classify(find_features(new_data_))
print("Tahmin Sonucu:", prediction)
print("Naive Bayes Algo Accuracy percent: ", (nltk.classify.accuracy(classifier, testing_set))*100)
#classifier.show_most_informative_features(15)

Tahmin Sonucu: Üçgen
Naive Bayes Algo Accuracy percent:  92.5925925925926


In [32]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_test)

prediction = MNB_classifier.classify(find_features(new_data_))
print("Tahmin Sonucu:", prediction)
print("MultinomialNB Algo Accuracy percent: ", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

Tahmin Sonucu: Üçgen
MultinomialNB Algo Accuracy percent:  89.81481481481481


In [33]:
BernaulliNB_classifier = SklearnClassifier(BernoulliNB())
BernaulliNB_classifier.train(training_test)

prediction = BernaulliNB_classifier.classify(find_features(new_data_))
print("Tahmin Sonucu:", prediction)
print("BernoulliNB Algo Accuracy percent: ", (nltk.classify.accuracy(BernaulliNB_classifier, testing_set))*100)

Tahmin Sonucu: Üçgen
BernoulliNB Algo Accuracy percent:  92.5925925925926


In [34]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC


LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_test)

prediction = LinearSVC_classifier.classify(find_features(new_data_))
print("Tahmin Sonucu:", prediction)
print("BernoulliNB Algo Accuracy percent: ", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

Tahmin Sonucu: Üçgen
BernoulliNB Algo Accuracy percent:  95.37037037037037


In [35]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_test)

prediction = LogisticRegression_classifier.classify(find_features(new_data_))
print("Tahmin Sonucu:", prediction)
print("LogisticRegression Algo Accuracy percent: ", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

Tahmin Sonucu: Üçgen
LogisticRegression Algo Accuracy percent:  94.44444444444444


In [36]:
SGDClassifier = SklearnClassifier(SGDClassifier())
SGDClassifier.train(training_test)

prediction = SGDClassifier.classify(find_features(new_data_))
print("Tahmin Sonucu:", prediction)
print("SGDClassifier Algo Accuracy percent: ", (nltk.classify.accuracy(SGDClassifier, testing_set))*100)

Tahmin Sonucu: Üçgen
SGDClassifier Algo Accuracy percent:  92.5925925925926


In [37]:
class VoteClassifier(ClassifierI):
  def __init__(self, *classifiers):
    self.classifiers = classifiers

  def classify(self, features):
    votes = []
    for c in self.classifiers:
      v = c.classify(features)
      votes.append(v)
    return mode(votes)

vote_classifier = VoteClassifier(classifier,
                                  MNB_classifier,
                                  BernaulliNB_classifier,
                                  SGDClassifier,
                                  LinearSVC_classifier,
                                  LogisticRegression_classifier)

prediction = vote_classifier.classify(find_features(new_data_))
print("Tahmin Sonucu:", prediction)
print("vote_classifier accuracy percent:", (nltk.classify.accuracy(vote_classifier, testing_set))*100)

Tahmin Sonucu: Üçgen
vote_classifier accuracy percent: 94.44444444444444
